# Digital Signature Algorithm (DSA)

In [1]:
import random
import hashlib

# Public Configuration

In [2]:
"""
# To generate random p and q pair for DSA
# !pip install pycryptodome
from Crypto.PublicKey import DSA
key = DSA.generate(3072)
p, q = key.p, key.q
"""

In [5]:
# p, q = (283, 47)
# p, q = (1279, 71)

p = int ("\
368577123415647035185869509923454362988806654876528082212642441963073\
112178399735415809071414511788930869249295250430304540353013866431229\
965510814733779933506798365268561425933870873229773800684661220325186\
508845233129736449679530102708242450177182372322415658482081901982139\
935504459436526193127136706104380369832924830561868635645974615813718\
599034288471386879791087503489121436698353515121613823867525619537313\
836546517502082093400007321208415057847562620627644914725375992993318\
465393374569764496785505998125381607827118352697037326000376764847745\
255637988916261264753020692214535700561224725217079718071094435237402\
156088273408028838936890398130926616753252644546343571080376158118499\
400126944433056814392717271382689271187098581742948664096320444706415\
422463846704028520445125935059579157543820582424507879000158982185479\
411941493007828836744389091928984640165167590618063453847542820383591\
5397282804819083435616816897")

q = 65032841498903519040222055260781303700863228372896251521604890600319447022433

In [6]:
assert (p - 1) % q == 0

In [7]:
a = int( (p - 1) // q )

In [8]:
print(f"p is {len(bin(p)[2:])} bits integer")
print(f"q is {len(bin(q)[2:])} bits integer")

p is 3072 bits integer
q is 256 bits integer


In [11]:
h = random.randint(2, p-2)

# calculate generator g
g = pow(h, a, p)

assert g > 1
assert pow(g, q, p) == 1

# Generating Keys

In [12]:
# private key of Alice
x = random.randint(1, q-1)

In [13]:
# public key of Alice
y = pow(g, x, p)

In [14]:
print(f"public key is {y}")

public key is 2537274542713973759728732535631537592622195502536814905848783605293501974712818948062042709423984685755965884261110863001262960999327779569027244173916711161068657094262221525949900005188481066809119183021560878577370201190785788970889364757835565474489891200612806867384355949686039069042382519897456040051835381732717848587733826232061898716634590959031831012776383485705753428279401258416396889787140267872761527084391094995435587013849391381591678233390182141655061173746054183533933270930921333164929134109090011976093405576423830798976442806126653189384280058234310177216652016516375387622671439586682498339645236050470993030677642041408239606122300224728498020607402457802664109971061162298868815684943461565590412380796352090113346471412524310480855627160595405151667613738149524193716644606684123592761128708751782536543830351917683863159703332022006363102143204624815663846813695252963884795672830187304378404399513


In [15]:
# public arguments: p, q, a, g

# Signing a message

In [16]:
def find_hash(m) -> int:
    
    if isinstance(m, int):
        m = str(m)
    
    # to bytes
    m = str.encode(m)
        
    hash_value = hashlib.sha1(m).digest()
    # Convert the hash value to an integer
    return int.from_bytes(hash_value, 'big') % q

In [17]:
# random key
k = random.randint(1, q-1)

In [18]:
r = pow(g, k, p) % q

In [19]:
message = "attack tomorrow!"

In [20]:
hash_value = find_hash(message)

In [21]:
s = ( pow(k, -1, q) * (hash_value + x * r) ) % q

In [22]:
assert r != 0 and s != 0

In [23]:
# signature
print(f"Signature of message '{message}' is (r={r}, s={s})")

Signature of message 'attack tomorrow!' is (r=24237691922726800337560444627135898348259583396094931661288502728197985066365, s=25656973189530970197943839068440842897634893307294833301781687664602365625445)


# Verification

Bob receives message, (r, s) pair as signature, y as public key of Alice and (p, q, a, g) public arguments.

In [24]:
hash_value = find_hash(message)

In [25]:
w = pow(s, -1, q)

In [26]:
u1 = (hash_value * w) % q

In [27]:
u2 = (r * w) % q

In [28]:
v = ( ( pow(g, u1, p) * pow(y, u2, p) ) % p ) % q

In [29]:
assert v == r

# Proof

Alice calculated s from the signature as 

s = k^-1 * (H + x * r) mod q

Find k from this equation

k = s^-1 * (H + x * r) mod q

k = H * s^-1 + x * r * s^-1 mod q

Multiplicative inverse of s was represented as w in the verification

k = c mod q

This equation can be used in the exponent of generator g

g^k = g^(H * w + x * r * w)

According to product rule of exponents, this can be represented as

g^k = g^(H * w) * g^(x * r * w)

According to power of a power rule, this can be represented as

g^k = g^(H * w) * (g^x)^(r * w)

G to the power of x is equal to the public key of Alice

g^k = g^(H * w) * y^(r * w)

We represented H*w as u1 and r*w as u2 in verification

g^k = g^u1 * y^u1

Alice calculated r from signature as g to the power of k

r = g^u1 * y^u1

Bob verifies this equation in the verification already!